In [ ]:
import pandas as pd
import numpy as np
import datetime

### Load the NCC Data

In [2]:
ncc = pd.read_csv('../data/NCC_2018_nutrients_per_100g_originalcolnames.txt',sep='\t')

Set Food ID as the index

In [3]:
ncc['Food ID'].is_unique

True

In [4]:
ncc = ncc.set_index('Food ID')

### Load the data from the ASA24 Recalls

In [5]:
asa_24 = pd.read_csv('../data/training_for_GS_122118.csv')

Set FoodCode as the index 

In [6]:
asa_24 = asa_24.set_index('FoodCode')

### Load the file listing matching columns between the datasets

In [7]:
matching = pd.read_csv('../data/matching_ncc_fndds_columns.txt',sep='\t')

Get the list of columns for each dataset

In [8]:
ncc_cols = matching['NCC.Term'].values.tolist()
asa_24_cols = matching['FNDDS.Term'].values.tolist()
asa_24_cols = [val.replace(" ","") for val in asa_24_cols]

### Calculate the pairwise correlations
Define a function to calculate the pairwise PCC matrix between two matrices A and B

In [9]:
def row_corr(A,B):
    #number of columns in A or B
    N = B.shape[1]

    # Store row-wise sums of A and B, as they would be used at few places
    sA = A.sum(1)
    sB = B.sum(1)

    # Compute the four terms in pcc matrix-wise
    p1 = N*np.einsum('ik,jk->ij',A,B)
    p2 = sB*sA[:,None]
    p3 = N*((B**2).sum(1)) - (sB**2)
    p4 = N*((A**2).sum(1)) - (sA**2)

    # compute pcc as 2D array 
    pcorr = ((p1 - p2)/np.sqrt(p3*p4[:,None]))
    return pcorr

Get the columns provided by the `matching` file

In [10]:
A = asa_24.loc[:,asa_24_cols].values
B = ncc.loc[:,ncc_cols].values

In [11]:
corr = row_corr(A,B)

/software/anaconda3/4.5.12/lssc0-linux/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
/software/anaconda3/4.5.12/lssc0-linux/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


### Get results

Get indices of the top matches from the correlation matrix

In [46]:
TOP = 5 # set the number of matches to return

indices_top = np.argsort(-corr,axis=1,)[:,:TOP]
pcc_top = np.sort(-corr,axis=1,)[:,:TOP]

In [45]:
indices_top.transpose()

array([[11197, 11213,  5474, ..., 13351,  5485,  5491],
       [16436, 11246, 11314, ..., 13347,  9773, 16046],
       [11198, 11245, 11312, ..., 13350, 10583, 13444],
       [11195,  8027, 11309, ..., 13357, 10580, 15110],
       [ 7102,  6772, 11311, ...,  9867,  5657,  4406]])

Construct dataframe to store top results

In [13]:
iters = [np.arange(0,asa_24.shape[0],1).tolist(),np.arange(0,TOP,1).tolist()]

results_cols = asa_24.columns.values.tolist() + ['PCC'] + ncc.columns.values.tolist()

mi = pd.MultiIndex.from_product(iters, names=['asa_index', 'match_index'])

results_top = pd.DataFrame(index=mi,columns=results_cols)

Copy ASA24 values to left side

In [58]:
results_top.loc[results_top.eval('match_index==0'), asa_24.columns] = asa_24.values

results_top.loc[:,ncc.columns] = ncc.iloc[indices_top.flatten(),:].values

results_top.loc[:,'PCC'] = -pcc_top.flatten()

### Save results to CSV

In [60]:
name = 'pcc_matching_results_top_{}'.format(TOP)
path = '../data/' + name
results_top.to_csv(path)

In [75]:
results_top_desc_only = results_top[['Food_Description','PCC','Keylist','Food Description','Short Food Description']]

In [77]:
name = 'pcc_matching_results_top_{}_desc_only'.format(TOP)
path = '../data/' + name
results_top_desc_only.to_csv(path)

In [80]:
results_top_desc_only

Food_Description  \
asa_index match_index                                                      
0         0            Milk, condensed, sweetened (formerly NS as to ...   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
1         0            Cocoa, whey, and low-calorie sweetener mixture...   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
2         0                                 Eggnog, made with whole milk   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
3         0                 Instant breakfast, powder, not reconstituted   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
4         0            Pudding, chocolate, ready-to-eat, low calorie,...   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
5         0                                                      Custard   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
...                                                                  ...   
372       0            Salty snacks, corn or cornmeal base, corn chip...   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
373       0                                   Cream substitute, powdered   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
374       0            Salty snacks, corn or cornmeal base, corn chip...   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
375       0            Coconut milk (liquid expressed from grated coc...   
          1                                                          NaN   
          2                                                          NaN   
          3                                                          NaN   
          4                                                          NaN   
376       0                 

In [3]:
pd.read_csv('../data/pcc_matching_results_top_5_desc_only')

,asa_index,match_index,Food_Description,PCC,Keylist,Food Description,Short Food Description
0,0,0,"Milk, condensed, sweetened (formerly NS as to ...",1.000000,MILK.CON.REG.REG,"milk, condensed (sweetened), regular, regular",Sweetened condensed milk
1,0,1,NaN,0.995898,SWEE.DUL,"sweeteners, dulce de leche (milk dessert or mi...",Dulce de leche
2,0,2,NaN,0.993710,MILK.CON.REG.RF,"milk, condensed (sweetened), regular, reduced fat","Sweetened condensed milk, reduced fat"
3,0,3,NaN,0.991908,MILK.CON.CHO,"milk, condensed (sweetened), chocolate","Sweetened condensed milk, chocolate"
4,0,4,NaN,0.985241,FAST.DQ.DES.BLI.HEA,"fast foods, DQ (Dairy Queen), desserts, Blizza...","Dairy Queen Blizzard, Heath"
5,1,0,"Cocoa, whey, and low-calorie sweetener mixture...",0.995726,MILK.GOA,"milk, goat","Milk, goat"
6,1,1,NaN,0.995076,MILK.MIX.CHO.RTD.NS.DEF,"milk, mixtures and milk drinks, chocolate, pur...","Chocolate milk, ready-to-drink, no sugar added"
7,1,2,NaN,0.995076,MILK.MIX.CHO.RTD.NS.COM.N50,"milk, mixtures and milk drinks, chocolate, pur...",Nestle Nesquik Ready-To-Drink - Chocolate Lowf...
8,1,3,NaN,0.994451,FAST.WEND.BEV.CHO,"fast foods, Wendy's, beverages, chocolate milk",Wendy's chocolate milk
9,1,4,NaN,0.994451,FAST.BURG.BEV.CHO,"fast foods, Burger King, beverages, chocolate ...",Burger King chocolate milk
